In [ ]:
filtered=sc.read('Capillaries_INCITE_object.h5ad')

In [ ]:
filtered.obs['did'] = filtered.obs['DiseaseState'].astype(str) + "_" + filtered.obs['SampleID'].astype(str) + filtered.obs['Sex'].astype(str)
# Compute the cell counts for each did
cell_counts = filtered.obs['did'].value_counts()

# Map these counts to the did column in adata.obs
filtered.obs['cell_count'] = filtered.obs['did'].map(cell_counts)


In [ ]:
filtered

In [ ]:
ind_count

In [ ]:
disease_colors <- c(
  "AD" = "#FF0000",      # Bright red
  "OldCtrl" = "#0000FF", # Bright blue
  "ALS" = "#D32F2F",     # Darker shade of red
  "YngCtrl" = "#4682B4", # Steel blue
  "FTD" = "#E57373"      # Light muted red
)


In [ ]:
sc.settings.set_figure_params(dpi=100)

all_ind_count = filtered.obs.groupby('SampleID').size().reset_index(name='all_count_sum')

for cluster in filtered.obs['final_leiden'].unique():
    # Subset data for current cluster
    c = filtered[filtered.obs["final_leiden"].isin([cluster])]
    
    # Compute counts for this cluster
    ind_count = c.obs.groupby(['SampleID', 'DiseaseState', 'final_leiden'], observed=True).size().reset_index(name='counts')
   # ind_count = ind_count[ind_count['DiseaseState'] != 'YngCtrl']
    ind_count['counts'] = ind_count['counts'].astype('float')
    mapped_series = ind_count['SampleID'].map(all_ind_count.set_index('SampleID')['all_count_sum'])
    mapped_series = mapped_series.astype('float64')

    ind_perc = (ind_count['counts'] / mapped_series) * 100
    # Add percentage column to the count data
    ind_count['cluster_perc'] = ind_perc
    
    # Merge with Age column
    ind_count = pd.merge(ind_count, filtered.obs[['SampleID', 'Age']], on='SampleID', how='left')
    
    # Convert 'Age' column to numeric
    ind_count['Age'] = pd.to_numeric(ind_count['Age'], errors='coerce')
    ind_count['Age'] = pd.to_numeric(ind_count['Age'], errors='coerce')
    ind_count['cluster_perc'] = pd.to_numeric(ind_count['cluster_perc'], errors='coerce')

    # Plot
    plt.figure()
    sns.regplot(x='Age', y='cluster_perc', data=ind_count, ci=99, scatter=False)  # Added ci=95 here
    ax = sns.scatterplot(x='Age', y='cluster_perc', hue='DiseaseState', data=ind_count, 
                         palette = ["#FF0000", "#D32F2F", "#E57373", '#0000FF', '#4682B4'], s=100)
    #'#5c8c44', '#a724f2', '#bd4dca', '#cc62cb', '#f7b363'
   # "#E63946", "#DAA520", "#A8DADC", "#457B9D", "#1D3557"
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.ylabel(f'Proportion of Cells in Cluster {cluster}')
    plt.xlabel('Age')
    plt.grid(False)
    plt.savefig(f'Cluster_{cluster}_vs_Age.png', bbox_inches='tight')
    
    # Spearman correlation
    correlation, p_value = stats.spearmanr(ind_count['Age'], ind_count['cluster_perc'].fillna(0))
    print(f"Cluster {cluster} - Spearman correlation:", correlation, "P-value:", p_value)
